In [1]:
import pickle
import pandas as pd
import os
from collections import defaultdict

In [8]:
#How complete checklists looks before & after Filtering Data for month of June 


In [9]:
with open('/network/scratch/t/tengmeli/scratch/ecosystem-embedding/ebird_data/L10000991_complete_checklists.pkl', 'rb') as f:
    data = pickle.load(f)
data=data[data['MONTH']=='June']
data['SPECIES_COMPLETE_CHECKLISTS'].isnull().any()

False

In [10]:
data.head()


,Month_Num,MONTH,SCIENTIFIC NAME,SAMPLING EVENT IDENTIFIER,SPECIES_COMPLETE_CHECKLISTS,MONTHWISE_COMPLETE_CHECKLISTS
3971,6,June,Acanthis flammea,NaN,0,2
3970,6,June,Acanthis hornemanni,NaN,0,2
3969,6,June,Accipiter cooperii,NaN,0,2
3968,6,June,Accipiter gentilis,NaN,0,2
3967,6,June,Accipiter striatus,NaN,0,2


In [11]:
len(data['SCIENTIFIC NAME'].unique())

684

In [12]:
#Reading training data & Finding  p(L) => Number of complete checklist in a state /total number of complete checklists

In [13]:
d=pd.read_csv('/network/scratch/t/tengmeli/scratch/ecosystem-embedding/training/train_clustered_vf.csv')
#finding p(L)
by_state=d.groupby(['state_id'])['num_complete_checklists'].sum()
pL=by_state/d['num_complete_checklists'].sum()
d['pL']=d['state_id'].map(pL)
d.head()

,Unnamed: 0,hotspot,r,g,b,nir,meta,rgb,species,env,ped,bioclim,hotspot_id,june_complete_checklists,state_id,num_complete_checklists,pL
0,0,L678935,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,L678935,15,12,432,0.006107
1,1,L1160919,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,L1160919,14,26,76,0.021957
2,2,L893164,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,L893164,8,4,107,0.109308
3,3,L210257,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,L210257,74,5,880,0.038515
4,4,L6223517,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,/network/scratch/t/tengmeli/scratch/ecosystem-...,L6223517,8,3,321,0.039159


In [14]:
#check values of pL 
#pL

In [15]:
#Mapping each state to its corresponding list of locations{'state_id':[hotspot1, hotspot2,...]}

In [16]:

state_to_locations=d.groupby('state_id').agg({'hotspot':list})
state_to_locations=dict(state_to_locations['hotspot'])
#state_to_locations[1]

In [17]:
#Finding Lookup Table speciesToState dictionary of dictionary =>how many checklists of that species in that state=> {species:{state:Num_checklists}}

In [ ]:
#Run Only Once 
#state_to_species mapping
species_to_state=defaultdict(lambda : defaultdict(lambda : 0))
for state,locations in state_to_locations.items():
    #print(locations)
    for location in locations:
        #print(location)
        path=os.path.join('/network/scratch/t/tengmeli/scratch/ecosystem-embedding/ebird_data',location + '_complete_checklists.pkl')
        with open(path, 'rb') as f:
               l_to_s = pickle.load(f)
        
        if  l_to_s.empty :
            print(f'cannot find file at {path}')
            continue
        #filter month of June for each location
        l_to_s=l_to_s[l_to_s['MONTH']=='June']
        #Preprocess N/A values by filling zeros=>some species have 0 occurance 
        l_to_s['SPECIES_COMPLETE_CHECKLISTS']=l_to_s['SPECIES_COMPLETE_CHECKLISTS'].fillna(0)
        assert len(l_to_s)==684 , 'length of filterd june is not as expected'
        assert not (l_to_s['SPECIES_COMPLETE_CHECKLISTS'].isnull().any() ), 'some null valus is present'
        for species in l_to_s['SCIENTIFIC NAME']:
            #print(species)
            #print(l_to_s[l_to_s['SCIENTIFIC NAME']==species]['SPECIES_COMPLETE_CHECKLISTS'])
            #break
            species_to_state[species][state]+=(l_to_s[l_to_s['SCIENTIFIC NAME']==species]['SPECIES_COMPLETE_CHECKLISTS'].to_numpy().item())
            
      

        

In [ ]:
#some checks
species_to_state['Acanthis hornemanni']
l_to_s[l_to_s['SCIENTIFIC NAME']=='Acanthis flammea']['SPECIES_COMPLETE_CHECKLISTS'].to_numpy().item()

In [ ]:
#Sanity checks
lookup_df=pd.DataFrame(species_to_state)
lookup_df.isnull().any()
lookup_df.head()

In [ ]:
#saving the lookup table 

with open('/network/scratch/t/tengmeli/scratch/ecosystem-embedding/training/species_to_states.pkl', 'wb') as f:
    pickle.dump(lookup_df, f)

In [ ]:
#Sanity checks=>sum species have 0 total occurance !
#sum(lookup_df['Vireo vicinior'])

In [ ]:
#a bar plot

In [ ]:
with open('/network/scratch/t/tengmeli/scratch/ecosystem-embedding/training/species_to_states.pkl', 'rb') as f:
    lookup_df= pickle.load(f)

lookup_df.plot.bar()


In [ ]:
#Calcultate probalities (p(L|y)) => num_checklists_reporting species y in state L /num of checklists reporting species y in all states 

lookup_df=lookup_df.apply(lambda x : x /sum(x))
lookup_df=lookup_df.fillna(0)
lookup_df['pL']=pL
lookup_df.head()

In [ ]:
#Saving 
with open('/network/scratch/t/tengmeli/scratch/ecosystem-embedding/training/species_to_states_probs.pkl', 'wb') as f:
    pickle.dump(lookup_df, f)

In [ ]:
#Correction factor => p(L|y)/p(L) 

In [ ]:
correction_factor=lookup_df.copy()
correction_factor.loc[:, correction_factor.columns!='pL']=correction_factor.loc[:, correction_factor.columns!='pL'].apply(lambda x : x/correction_factor['pL'])

In [ ]:
#check how it looks
correction_factor

In [ ]:
#some statistics 
correction_factor.describe()

In [ ]:
correction_factor.max().max()

In [ ]:
correction_factor.min().min()

In [ ]:
correction_factor.mean().mean()

In [ ]:
#saving correction factor

In [ ]:

with open('/network/scratch/t/tengmeli/scratch/ecosystem-embedding/training/correction_factor.pkl', 'wb') as f:
    pickle.dump(correction_factor, f)

In [4]:
with open('/network/scratch/t/tengmeli/scratch/ecosystem-embedding/training/correction_factor.pkl', 'rb') as f:
    df= pickle.load(f)

In [5]:
df

,Acanthis flammea,Acanthis hornemanni,Accipiter cooperii,Accipiter gentilis,Accipiter striatus,Acridotheres tristis,Actitis macularius,Aechmophorus clarkii,Aechmophorus occidentalis,Aegolius acadicus,...,Vireo vicinior,Xanthocephalus xanthocephalus,Xema sabini,Zenaida asiatica,Zenaida macroura,Zonotrichia albicollis,Zonotrichia atricapilla,Zonotrichia leucophrys,Zonotrichia querula,pL
1,0.0,0.0,0.680117,0.000000,0.000000,0.000000,0.088549,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.630401,1.285975,0.000000,0.000000,0.000000,0.000000,0.002268
2,0.0,0.0,1.095615,0.000000,0.000000,0.000000,0.142645,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.072469,0.000000,0.000000,0.000000,0.000000,0.002816
3,0.0,0.0,1.694104,0.856944,0.383693,0.000000,0.066683,0.306750,0.232346,0.354679,...,1.233668,0.545962,0.000000,8.468486,0.803282,0.000000,0.000000,0.066569,0.000000,0.039159
4,0.0,0.0,1.955429,1.780567,0.702551,0.000000,0.547601,5.269276,3.546372,2.668292,...,0.265172,0.818627,4.574215,0.388543,0.964981,0.000000,3.049476,2.870923,0.000000,0.109308
5,0.0,0.0,1.329154,6.098932,2.730774,0.000000,2.385968,1.075987,3.808344,2.884892,...,7.776665,3.590198,0.000000,0.055693,1.306010,0.046447,0.000000,1.358855,0.000000,0.038515
6,0.0,0.0,0.417333,0.676961,2.525890,0.000000,0.526775,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.277863,0.126310,0.000000,0.000000,0.000000,0.019828
8,0.0,0.0,0.266727,0.000000,0.000000,0.000000,0.101865,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.883696,0.000000,0.000000,0.000000,0.000000,0.007887
9,0.0,0.0,0.480459,0.000000,0.000000,26.763057,0.040318,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.336787,0.856014,0.000000,0.000000,0.000000,0.000000,0.037365
10,0.0,0.0,1.097024,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.873118,0.000000,0.000000,0.000000,0.000000,0.013808
11,0.0,0.0,0.574503,0.000000,0.000000,0.000000,1.172460,0.000000,0.169405,0.000000,...,0.000000,0.282497,0.000000,0.000000,1.313869,0.000000,0.000000,0.000000,0.000000,0.004883
